# First steps into Bayesian optimization
*Ivo Couckuyt*, *Joachim van der Herten*

## Introduction

Bayesian optimization is particularly useful for expensive optimization problems. This includes optimization problems where the objective (and constraints) are time-consuming to evaluate: measurements, engineering simulations, hyperparameter optimization of deep learning models, etc. Another area where Bayesian optimization may provide a benefit is in the presence of (a lot of) noise. If your problem does not satisfy these requirements other optimization algorithms might be better suited.

To setup a Bayesian optimization scheme with GPflowOpt you have to:

- define your objective and specify the optimization domain
- setup a GPflow model and choose an acquisition function
- create a BayesianOptimizer

## Objective function

In [1]:
import numpy as np
from gpflowopt.domain import ContinuousParameter


def fx(X):
    X = np.atleast_2d(X)
    return np.sum(np.square(X), axis=1)[:, None]

domain = ContinuousParameter('x1', -2, 2) + ContinuousParameter('x2', -1, 2)
domain

/home/javdrher/.virtualenvs/gpflowopt-nextgen/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Name,Type,Values
x1,Continuous,[-2. 2.]
x2,Continuous,[-1. 2.]


## Bayesian optimizer

In [3]:
import gpflow
from gpflowopt.bo import BayesianOptimizer
from gpflowopt.design import LatinHyperCube
from gpflowopt.acquisition import ExpectedImprovement
from gpflowopt.optim import SciPyOptimizer

# Use standard Gaussian process Regression
lhd = LatinHyperCube(21, domain)
X = lhd.generate()
Y = fx(X)

with gpflow.defer_build():
    model = gpflow.models.GPR(X, Y, gpflow.kernels.Matern52(2, ARD=True))
    model.kern.lengthscales.transform = gpflow.transforms.Log1pe(1e-3)
model.compile()

# Now create the Bayesian Optimizer
alpha = ExpectedImprovement(model)
optimizer = BayesianOptimizer(domain, alpha)

# Run the Bayesian optimization
with optimizer.silent():
    r = optimizer.optimize(fx, n_iter=10)
print(r)

INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 12.494610
  Number of iterations: 6
  Number of functions evaluations: 9
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: -24.872416
  Number of iterations: 75
  Number of functions evaluations: 94
INFO:tensorflow:Optimization terminated with:
  Message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
  Objective function value: -30.707543
  Number of iterations: 0
  Number of functions evaluations: 21
INFO:tensorflow:Optimization terminated with:
  Message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
  Objective function value: -30.707543
  Number of iterations: 0
  Number of functions evaluations: 21
INFO:tensorflow:Optimization terminated with:
  Message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
  Objective function value: -37.430311
  Number of iterations: 3
  Number of functions evaluations: 65
I

That's all! Your objective function has now been optimized for 15 iterations.